# Data_Cleansing
#### Author: Armin Berger

Date: 03/10/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

Libraries used: 
* re 
* os 
* pandas 
* numpy 
* nltk.sentiment.vader SentimentIntensityAnalyzer
* haversine, Unit
* math
* sklearn.linear_model LinearRegression
* matplotlib


### Overview:

This project contains three main parts.

- Wrangling dirty_data
- Removing outliers
- Imputing missing values 

These three core tasks are underlined in red to help the reader. Moreover, each task is devided into enumerated subtasks. 

# Import libraries 

Firstly, we will import all the required libraries

In [ ]:
# load the required libraries for this assignment 

import pandas as pd # used for dataframe maniulation and general wrangling

import numpy as np # used for mathematical operations

import re # used for regular expression

import os # used for reading in multiple file from the local os 

from nltk.sentiment.vader import SentimentIntensityAnalyzer # used to get sentiment of text

import math # used for linear operations

from sklearn.linear_model import LinearRegression # used for linear regression models

from matplotlib import pyplot as plt # used for plotting histograms and boxplots

# <span style="color:red"> 1. Wrangling dirty_data

In the first part of this project we read in the csv file '26255367_dirty_data.csv' and remove both syntactical and semantic anomalies/erros.
Firtly, we will detect and remove all syntactical anomalies. Following that, all semantic erros will be removed. For both steps we will start with a 'dirty dataframe' and then over the course of identifying and fixing errors in rows we will add these rows to a 'clean dataframe'. This is possible since it was stated in the assignment brief, that each row can only containe a maximum of one syntactic or semantic error. Thus, if an error is fixed in a certain row that row can't have any further anomalies and can be added to the 'clean dataframe'.

## <span style="color:blue"> A. Identify and Remove Syntactical Anomalies 
    
In this step we will fix all errors that are realted to format issue (syntactic). We will go through the dataframe column by column, starting with the order_id. 

### A.1 Read in Data

In [ ]:
# read in our dirty data in csv format
dirty_data_df = pd.read_csv('26255367_dirty_data.csv')


In [ ]:
# display first 5 rows to get an idea of the structure and type of data
dirty_data_df.head()


In [ ]:
# check how many rows and columns the df has
dirty_data_df.shape


In [ ]:
# shows information of all columns
dirty_data_df.info()


Using the info() command we can see that there are no null values in the entire data set

### A.2 Check if order_ids are all unique 

In [ ]:
# since each order is unique, there should be no duplicate order_ids 
# we use the describe function to check for duplciates
# only select columns with data type object (string like data type)
dirty_data_df.describe(include=['O'])


After using the command .describe(include=['O']) we could see that there we 500 unique order_ids, thus no duplicates need to be removed. Futhermore, it could be obserevd that the dataset contains some duplicate customer_ids. This is unproblematic, however, since the same customer could palce multiple unique order, thus no actions are required for the customer_id as well.

### A.3 Check if the dates are in correct format (YYY-MM-DD)

In [ ]:
# we seek to detect and isolate all date values that don't match our pattern of YYYY-MM-DD
# done with the to_datetime function

# scan through the date column of all rows
# in case the date format doesn't fit our desired format we set the row in date_mask = True 
date_mask = pd.to_datetime(dirty_data_df.date, format='%Y-%m-%d', errors='coerce').isna()

# now we extract all the rows which have an index matching the index value saved in date_mask
# the loc[] function helps us filter out these rows and save them in dirty_date
dirty_date = dirty_data_df.loc[date_mask, :]

# check how many rows have a faulty date
dirty_date.shape

In [ ]:
# this data is all the columns that have faulty date formats
# this 'visual analysis' enables us to check for the kind of date format issues we need to fix
dirty_date.head(27)

In [ ]:
# after selecting all rows with a faulty date entry we need to correct the date format
# for that purpose we have created a function

def date_change(wrong_date):
    
    # we split the date on -
    pieces_date = wrong_date.split('-')
    
    # set the year  
    year = (pieces_date[0])
    
    # set the month  
    month = (int)(pieces_date[1])
    
    # set the days of each month
    day = (int)(pieces_date[2]) 
    
    # check if month and year were switched
    # if yes we swap them
    if len(year) != 4:
        year, day = day,(int)(year)
    
    # check if the month is larger than 12
    # if yes than switch month for days
    if month > 12:
        day, month = month,day
    
    # check if months with 30 days have more than 30 days
    # if yes than set them to 30 days 
    if month in (4,6,9,11):
        if day > 30:
            day = 30
    
    # check the month of february
    # set days to 28 incase the given number is large than that
    if month == 2:
        if day > 28:
            day = 28
    
    
    # now reconstruct the deconstructed date 
    # we combine the year, month, and day as strings and save them as a proper date fromat
    formatted_date = pd.to_datetime(str(year)+"-"+str(month)+"-"+str(day),format='%Y-%m-%d')
    
    # return the edited dates
    return formatted_date

In [ ]:
# using the function date_change to edit all the selected dates in faulty format
dirty_data_df['date'] = dirty_data_df['date'].apply(date_change)

In [ ]:
# re-using the same code form above we check if there are any dates left in faulty format
date_mask = pd.to_datetime(dirty_data_df.date, format='%Y-%m-%d', errors='coerce').isna()
dirty_date = dirty_data_df.loc[date_mask, :]

# check how many rows still have faulty dates left
dirty_date.shape

### A.4 Check if the nearest_warehouse values are spelled correctly

In [ ]:
# read in the names of the three warehouse 
warehouse_df = pd.read_csv('warehouses.csv')

# display the names of the three warehouses
warehouse_df['names']

In [ ]:
# check the spelling of the nearest_warehouse names 
warehouse_count = dirty_data_df['nearest_warehouse'].value_counts()

# display the value counts
warehouse_count

As you can see the same warehouse is not always spelt consistently, thus we need to set them to a consistent spelling.  

In [ ]:
# we need a function to edit and uniform all the nearest_warehouse names 
# we fix nearest_warehouse names based on the starting letter of nearest_warehouse
def fix_nearest_warehouse(warehouse):
    
    # if a word starts with the letter b
    if warehouse.startswith('b') or warehouse.startswith('B') :
        
        # we return the correctly spelled name of warehouse 'Bakers'
        return 'Bakers'
    
    # if a word starts with the letter n
    elif warehouse.startswith('n') or warehouse.startswith('N'):
        
        # we return the correctly spelled name of warehouse 'Nickolson'
        return 'Nickolson'
    
    # if a word starts with the letter t
    elif warehouse.startswith('t') or warehouse.startswith('T'):
        
        # we return the correctly spelled name of warehouse 'Thompson'
        return 'Thompson'
    

In [ ]:
# now we go through all the values for nearest_warehouse and fix their spelling
dirty_data_df['nearest_warehouse'] = dirty_data_df.apply(lambda row:fix_nearest_warehouse(row['nearest_warehouse']),axis=1)

In [ ]:
# final check if transformation worked and warehouses have uniform names 
warehouse_count = dirty_data_df['nearest_warehouse'].value_counts()
warehouse_count


Now all the warhouses have uniform names.

### A.5 Check if items in shopping_cart are uniformly spelled


In [ ]:
# we seek to check if all the individual items in the shopping_cart column are spelled uniformly
# catch all the individual items using regular expression and save them in a list
lst = list(dirty_data_df['shopping_cart'].apply(lambda x: re.findall("\(\'(.+?)\', ",x)))

In [ ]:
# now that we have all the individual items in a list of lists we seek to merge them and only keep the unique ones
# using list comprehension unpack the list of lists and save the output as a set 
unique_items = set([item for lists in lst for item in lists])

# display the unique items
unique_items

Output shows that there are 10 unique items in the column shopping_cart, thus there are no duplicates or missspellings to fix up.

### A.6 Checking customer_lat and customer_long

In [ ]:
# check the longitude values using the describe() methode
long_count = dirty_data_df['customer_long'].describe()

# display the statistics
long_count

In [ ]:
# plot the longitude values
dirty_data_df.hist(column='customer_long')

We can observe that there are incorrect longitude values.

In [ ]:
# check the latitude values using the describe() methode
lat_count = dirty_data_df['customer_lat'].describe()

# display the statistics
lat_count

In [ ]:
# plot the latitude values
dirty_data_df.hist(column='customer_lat')

We can observe that there are incorrect longitude values.

In [ ]:
# melbourne has a latitude of -37.840935 and a longitude of 144.946457
# when skimming over the data we saw customer_lat values starting with 144 and customer_long values starting with
# -37, which clearly indicates that they are interchanged
# therefore we need to find a way to swap them 

# we identify all the rows in which the longitude and latitude were mixed up and save them in a sub data frame
interchanged_df = dirty_data_df[(dirty_data_df['customer_lat'] > 0) & (dirty_data_df['customer_long'] < 0)]

# create a series in which we can save whether in a row the longitude and latitude was mixed up
interchange_idx = (interchanged_df['customer_lat'] > 0) & (interchanged_df['customer_long'] < 0)

# check the rows where longitude and latitude need to be fixed
interchanged_df.head()

In [ ]:
# the rows in which the lat and long of customer's are interchanged, are now being swapped 
# the function .loc only takes the rows where interchange_idx matches 
interchanged_df.loc[interchange_idx,['customer_lat','customer_long']] = interchanged_df.loc[interchange_idx,\
                                                                       ['customer_long','customer_lat']].values

# display the top 10 rows of the interchanged_df to see if the swapping worked
interchanged_df.head(10)

In [ ]:
# after swapping longitude and latitude in all the rows where they were mixed up we now check
# if longitude and latitude values have undesired + or - signs 

# ~ is used for filtering, it only keep the rows where the latitude and lonitude were not changed up
dirty_data_df_new = dirty_data_df[~dirty_data_df['order_id'].isin(interchanged_df['order_id'])]

In [ ]:
# create a function that checks whether lat is larger than 0
# required to get correct latitude values 
def check_lat(lat):
    
    if lat > 0:
        
        # since australia is in the southern hemisphere the latitude values need to
        # be negative
        lat = -lat
    
    # return correct lat
    return lat

In [ ]:
# now we apply the check_lat() function to the remanining part of the dataframe 
dirty_data_df_new.loc[:,'customer_lat'] = dirty_data_df_new.apply(lambda row:check_lat(row['customer_lat']),axis=1)

In [ ]:
# create a function that checks whether lon is smaller than 0
# required to get correct longitude values 
def check_lon(lon):
    
    if lon < 0:
        
        # since australia only has positive longitude values, any negative value needs to
        # be turned positive using abs()
        lon = abs(lon)
    
    # return correct lon
    return lon

In [ ]:
# now we apply the check_lon() function to the remanining part of the dataframe 
dirty_data_df_new.loc[:,'customer_long'] = dirty_data_df_new.apply(lambda row:check_lon(row['customer_long']),axis=1)

In [ ]:
# in the end we combine the two dataframes from the 
dirty_data_df = pd.concat([interchanged_df , dirty_data_df_new])

# check if we have all columns in one dataframe again
dirty_data_df.shape

Now we do a last check if we were able to successfully fix up the customer_lat and customer_long columns.

In [ ]:
# display statistics of 'customer_lat'
dirty_data_df['customer_lat'].describe()

In [ ]:
# display statistics of 'customer_long'
dirty_data_df['customer_long'].describe()

### A. 7 Checking the spelling of season

In [ ]:
# check the spelling of the seasons
season_count = dirty_data_df['season'].value_counts()

# display the count of seaon values
season_count

As this shows we need to standardise season names.

In [ ]:
# we need a function to edit and standardise all the season names
# we fix season names based on the starting letter of the seaosn
def fix_season(season_input):
    
    # set season input string to lower case
    season_input = season_input.lower()
    
    # in case string input starts with 'au.' return 'Autumn'
    if re.search('au.',season_input) :
        
        # return correct season name
        return 'Autumn'
    
    # in case string input starts with 'su.' return 'Summer'
    elif re.search('su.',season_input) :
        
        # return correct season name
        return 'Summer'
    
    # in case string input starts with 'sp.' return 'Spring'
    elif re.search('sp.',season_input) :
        
        # return correct season name
        return 'Spring'
    
    # in case string input starts with 'wi.' return 'Winter'
    elif re.search('wi.',season_input) :
        
        # return correct season name
        return 'Winter'
    

In [ ]:
# now we go through all the cloumn values for season and fix their spelling
dirty_data_df['season'] = dirty_data_df.apply(lambda row:fix_season(row['season']),axis=1)

In [ ]:
# check the spelling of the nearest_warehouse names 
season_count = dirty_data_df['season'].value_counts()

# display the season values to afirm their uniformity
season_count

### A. 8 Checking expedited delivery 

In [ ]:
# check if expedited delivery has correct boolean variables
expedited_count = dirty_data_df['is_expedited_delivery'].value_counts()

# display values
expedited_count

The column is_expedited_delivery only has two boolean values (True/False), thus no changes need to be made. 

### A.9 Checking is_happy_customer

In [ ]:
# check if is_happy_customer has correct boolean variables
is_happy_customer_count = dirty_data_df['is_happy_customer'].value_counts()

# display values
is_happy_customer_count

The column is_happy_customer only has two boolean values (True/False), thus no changes need to be made. 

## <span style="color:blue"> B. Identify and Remove Semantic Anomalies 

After identifying all syntactic errors we will now focus on fixing up all semantic errors.

In [ ]:
# list to save the index value of all rows that were already fixed up
# this is useful since all rows can only have one semantic error
fixed_columns = []

### B.1 Check if season value matches with date

In [ ]:
# check is all orders follow correct order time, that don't match the specified condition
# to check whether the season matches with the actual date we created we created 
# the function fix_season() 
# assigns a season given the date
def check_season(date, season):
    
    # turn input into string type
    date = str(date)
    
    # we split the date on -
    pieces_date = date.split('-')
    
    # save the month value as an int
    month = (int)( pieces_date[1])
    
    # spring is month 09-11
    if month > 8 and month < 12 and season == 'Spring':
        return True
        
    # summer is month 12-02
    elif month == 12 or month == 1 or month == 2 and season == 'Summer':
        return True
    
    # autumn is month 3-5
    elif month > 2 and month < 6 and season == 'Autumn':
        return True
    
    # winter is month 6-8
    elif month > 5 and month < 9and season == 'Winter':
        return True
    
    # if any of the above defined cases don't match return False
    else:
        return False

In [ ]:
# function that assigns a season given the date provided for each row
# fixes faulty dates
def fix_season(date):
    
    # turn input into string type
    date = str(date)
    
    # we split the date on -
    pieces_date = date.split('-')
    
    # save the month value as an int
    month = (int)( pieces_date[1])
    
    # spring is month 09-11
    if month > 8 and month < 12:
        season = 'Spring'
        
    # summer is month 12-02
    elif month == 12 or month == 1 or month == 2:
        season = 'Summer'
    
    # autumn is month 3-5
    elif month > 2 and month < 6:
        season = 'Autumn'
    
    # winter is month 6-8
    elif month > 5 and month < 9:
        season = 'Winter'
    
    # return correct season value
    return season

In [ ]:
# select all the rows with faulty season values given the date provided
season_false = dirty_data_df.apply(lambda row: check_season(row['date'], row['season']),axis=1)

In [ ]:
# append the index value of all the rows with faulty season values given the date provided to the list
# fixed_columns
[fixed_columns.append(x) for x in list(dirty_data_df.index[season_false == False])]

In [ ]:
# apply the function to all rows for the column oder_type
# since we need two columns both the order type and time we use a lambda within the apply function
dirty_data_df['season'] = dirty_data_df.apply(lambda row:fix_season(row['date']),axis=1)

# manuly checking if our manipulation was succesfull
dirty_data_df.head()

### B.2 Check if customer was satisfied with most recent order

In order to assess whether a customer was satisfied with most recent order or not we will use the SentimentIntensityAnalyzer() that is part of nltk. Using SentimentIntensityAnalyzer() we generate a polarity_scores that is saved in a seperate column of our main dataframe dirty_data_df. 

In [ ]:
# check the spelling of the nearest_warehouse names 
is_happy_customer_count = dirty_data_df['is_happy_customer'].value_counts()

# display values 
is_happy_customer_count

In [ ]:
# create a function that creates polarity_scores

def get_sentiment(input):
    
    # save the SentimentIntensityAnalyzer()
    analyzer = SentimentIntensityAnalyzer()

    # calculate polarity score
    sentiment = analyzer.polarity_scores(input)
    
    # return calculated polarity score
    return sentiment

In [ ]:
# function to check whether the is_happy_customer boolean values matches the sentiment scores calculated 
def check_happy(score,is_happy):
    
    # scores larger or equal to 0.05 are indicative of a postive sentiment
    # thus the is_happy variable should be positive
    if score  >= 0.05 and is_happy == True: 
        
        # if score = postive and is_happy = true we return a True value
        return True

    # scores smaller than 0.05 are indicative of a negative sentiment
    # thus the is_happy variable should be negative
    elif score < 0.05 and is_happy == False:
        
        # if score = negative and is_happy = false we return a True value
        return True
    
    # in all other cases we return a False value
    else:
        return False

In [ ]:
# function to fix the is_happy_customer boolean values 
def fix_happy(score):
    
    # return happy True if sentiment_score >= 0.05
    if score  >= 0.05: 
        return True
    
    # return happy False in all other case
    else:
        return False

In [ ]:
# apply the function get_sentiment to the column latest_customer_review to calculate a sentiment score
dirty_data_df['sentiment_score'] = dirty_data_df.apply(lambda row: get_sentiment(row['latest_customer_review'])["compound"],axis=1)

In [ ]:
# checking the sentiment_scores we just calculated
# as the histogram shows most of the reviews have posituive sentiment scores
dirty_data_df.hist(column='sentiment_score')

In [ ]:
# seletc and save the rows in which the sentiment score and is_happy_customer didn't match 
correct_happy_bool = dirty_data_df.apply(lambda row: check_happy(row['sentiment_score'],row['is_happy_customer']),axis=1)

In [ ]:
# append the index values of these selected rows to the list fixed_columns
[fixed_columns.append(x) for x in list(dirty_data_df.index[correct_happy_bool == False])]

In [ ]:
# after identifying all the misslabelings in the is_happy_customer column
# we fix them by applying the fix_happy methode to dirty_data_df['is_happy_customer'] column
dirty_data_df['is_happy_customer'] = dirty_data_df.apply(lambda row: fix_happy(row['sentiment_score']),axis=1)

In [ ]:
# lastly check if we succesfully removed all misslabelings in the is_happy_customer column
correct_happy_bool = dirty_data_df.apply(lambda row: check_happy(row['sentiment_score'],row['is_happy_customer']),axis=1)

# display the values
correct_happy_bool.value_counts()

The is_happy_customer column is successfully fixed.

### B.3 Check if nearest_warehouse values are really the nearest warehouse


It will be assessed whether the values in column nearest_warehouse are actually the the nearest warehouses to each
customer. This is done by using the read in coordinate information of of the warehouses and compare them with the customer's coordinates. The we will calculate the distance to each warehouse and choose the shortest distance.

In [ ]:
# read in the names and locaction of the three warehouse 
warehouse_df = pd.read_csv('warehouses.csv')

In [ ]:
# check lat and lon values of the three warehouses
warehouse_df.head()

In [ ]:
dirty_data_df['nearest_warehouse'].value_counts()

In [ ]:
# function to calculate distance to nearest warehouse
def cal_distance(warehouse, customer):
    
    # save the coordinates required to calculate the distance 
    lat_1, long_1 = warehouse
    lat_2, long_2 = customer

    # save the radius of the earth in m
    radius = 6378000  
    phi_1 = math.radians(lat_1)
    phi_2 = math.radians(lat_2)

    # haversine calculations required to determine distance to nearest warehouse
    delta_phi = math.radians(lat_2 - lat_1)
    delta_lambda = math.radians(long_2 - long_1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # the distance in meters 
    meters = radius * c 
    
    # the distance in kilometers
    km = meters / 1000.0
    
    # rounding the meter and km output
    meters = round(meters, 4)
    km = round(km, 4)
    
    # return the final rounded distance to warehouse in km
    return km

In [ ]:
# create methode that calaculates the distance to the nearest warehosue and retuns the 
# name of that warehouse 
def compare_dist_warehouse(cust_lat, cust_lon):
    
    # save customer postion as  tuple
    cust_cor = (cust_lat, cust_lon)
    
    # list of distances to warehouse
    dist_warehouse_list = []
    
    ## First, get all warehouse locations
    
    # save location of 'Nickolson' warehouse as tuple (lon,lat)
    warehouse_loc_nick = (warehouse_df.loc[0,'lat'], warehouse_df.loc[0,'lon'])
    
    # save location of 'Thompson' warehouse as tuple (lon,lat)
    warehouse_loc_thomp = (warehouse_df.loc[1,'lat'], warehouse_df.loc[1,'lon'])
    
    # save location of 'Bakers' warehouse as tuple (lon,lat)
    warehouse_loc_baker = (warehouse_df.loc[2,'lat'], warehouse_df.loc[2,'lon'])
    
    
    ## Calculate the distance from the customer to each warehouse
    
    # distance from customer to warehouse 'Nickolson'
    dist_to_nick =  float(cal_distance(warehouse_loc_nick, cust_cor))
    
    # append calculated distance to list 
    dist_warehouse_list.append((dist_to_nick, 'Nickolson'))
    
    # distance from customer to warehouse 'Thompson'
    dist_to_thomp =   float(cal_distance(warehouse_loc_thomp, cust_cor))
    
    # append calculated distance to list 
    dist_warehouse_list.append((dist_to_thomp, 'Thompson'))
    
    # distance from customer to warehouse 'Bakers'
    dist_to_baker =   float(cal_distance(warehouse_loc_baker, cust_cor))
    
    # append calculated distance to list 
    dist_warehouse_list.append((dist_to_baker, 'Bakers'))
    
    # reverese sort the tuples in the list base on the frist value (distance)
    # in each value 
    dist_warehouse_list.sort(key = lambda x: x[0])
    
    # return the first item of the list, which is the warehouse with the shortest distance 
    
    return dist_warehouse_list[0][1]
 

In [ ]:
# calculate the warhouse which is the nearest warehouse to the customer
dirty_data_df['nearest_warehouse'] = dirty_data_df.apply(lambda row: compare_dist_warehouse(row['customer_lat'],row['customer_long']),axis=1)

In [ ]:
dirty_data_df['nearest_warehouse'].value_counts()

### B.4 Check if distance_to_nearest_warehouse is correct given the provided warehouse value

It will be assessed whether the column distance_to_nearest_warehouse matches the actual distance to the declared warehouse.

#### Assess whether column distance_to_nearest_warehouse has any errors and fix them

In [ ]:
# calculation haversine procedure was adapted from 
# https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula

# function to calculate distance to nearest warehouse
def cal_warehouse_dist(warehouse, lat, lon):
    
    # check which warehouse is nearest to customer and save the coordinates 
    if warehouse == 'Nickolson':
        warehouse_loc = (warehouse_df.loc[0,'lon'],warehouse_df.loc[0,'lat'])
    
    elif warehouse == 'Thompson':
        warehouse_loc = (warehouse_df.loc[1,'lon'],warehouse_df.loc[1,'lat'])
        
    elif warehouse == 'Bakers':
        warehouse_loc = (warehouse_df.loc[2,'lon'],warehouse_df.loc[2,'lat'])

    # save the coordinates required to calculate the distance 
    long_1, lat_1 = warehouse_loc
    long_2, lat_2 = (lon, lat)

    # save the radius of the earth in m
    radius = 6378000  
    phi_1 = math.radians(lat_1)
    phi_2 = math.radians(lat_2)

    # haversine calculations required to determine distance to nearest warehouse
    delta_phi = math.radians(lat_2 - lat_1)
    delta_lambda = math.radians(long_2 - long_1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # the distance in meters 
    meters = radius * c 
    
    # the distance in kilometers
    km = meters / 1000.0
    
    # rounding the meter and km output
    meters = round(meters, 4)
    km = round(km, 4)
    
    # return the final rounded distance to warehouse in km
    return km

In [ ]:
# function that checks whether the given distance and calculated distance match
# if yes return True if not then return False  
def check_if_warehouse_dist_cor(given_distance,correct_distance):
    
    # check for the difference between the given_distance and the correct_distance
    # we used <= 0.01 to provide a small margin of error
    if float(given_distance) - float(correct_distance) <= 0.01:
        
        # if distance is almost, is the same return True
        return True
    
    # in all other cases we return False
    else:
        return False

In [ ]:
# function that checks whether the given distance and calculated distance match
# if not then replace given distance with calculated distance 
def fix_dist_to_nearest_warehouse(given_distance,correct_distance):
    
    # ensure that input is in float format
    given_distance = float(given_distance)
    correct_distance = float(correct_distance)
    
    # check is given_distance is unequal correct_distance
    # if not return correct_distance
    if given_distance - correct_distance > 0.01:
        
        return correct_distance
    
    # if almost equal to correct_distance return given_distance
    else:
        
        return given_distance
        

In [ ]:
# calculate the correct distance from a customer to the nearest warehouse 
dirty_data_df['cor_distance_to_nearest_warehouse'] = dirty_data_df.apply(lambda row: cal_warehouse_dist(row['nearest_warehouse'],row['customer_lat'],row['customer_long']),axis=1)

In [ ]:
# check if the provided distance from a customer to the nearest warehouse is correct
# for all incorrect distances, row index is saved 
warehouse_dist_false_true = dirty_data_df.apply(lambda row: check_if_warehouse_dist_cor(row['distance_to_nearest_warehouse'],row['cor_distance_to_nearest_warehouse']),axis=1)

In [ ]:
# append the index values of these incorrect distances to the list fixed_columns list
[fixed_columns.append(x) for x in list(dirty_data_df.index[warehouse_dist_false_true == False])]

In [ ]:
# now we fix all incorrect distances by replacing the incorrect given distance by replacing it with
# our calculateddistance
dirty_data_df['distance_to_nearest_warehouse'] = dirty_data_df.apply(lambda row: fix_dist_to_nearest_warehouse(row['distance_to_nearest_warehouse'],row['cor_distance_to_nearest_warehouse']),axis=1)

In [ ]:
# final check if given distance is equal to correect distance 
dirty_data_df.apply(lambda row: check_if_warehouse_dist_cor(row['distance_to_nearest_warehouse'],row['cor_distance_to_nearest_warehouse']),axis=1).value_counts()

The distance_to_nearest_warehouse values are all correct now.

### B.5 Calculate the price of each shopping cart item and check the order price

In order to check whether the order price is correct we need to determine the price of individual items. That can be done using a set of multilinear equations.

In [ ]:
# in order to solve a multilinear equation we need reliable data
# thus we need to use rows in which we already determined and
# fixed semantic erros, the index of these rows are saved in fixed_columns
correct_rows = dirty_data_df.loc[fixed_columns]

# only keep the shopping cart and order price
correct_rows = correct_rows[['shopping_cart','order_price']]


In [ ]:
# function that only keeps the data needed of each shopping cart row entry
# which is product name and quantity
def clean_input(input):
    
    # save input in string format
    input = str(input)
    
    # list to save ordered items and their quantity in
    order_list = list()
    
    # split the input base on '),' since each shopping cart item is sperated by () and a ,
    input = input.split('),')
    
    # iterat through the list of items
    for i in input:
        
        # find the name of the item
        item = re.findall(r'\'(.+)\',', i)
        
        # find it's quantity
        quantity = re.findall(r'\'.+\',.([0-9])', i)
        
        # save the item and it's quantity
        part = (item[0],int(quantity[0]))
        
        # append items to the order_list
        order_list.append(part)
    
    # return final order list
    return order_list
        

In [ ]:
# apply the function clean_input to all rows to rows 
# this returns an array of items and their quantity ordered which is saved in items_clean_series
items_clean_series = correct_rows.apply(lambda row: clean_input(row['shopping_cart']) ,axis=1)

In [ ]:
# reset the index values of the saved array of shopping items
correct_rows.reset_index(drop = True, inplace =True)

In [ ]:
# create an empty dataframe to append our shopping items and their quantity to
items_matrix = pd.DataFrame(columns=list(unique_items))

In [ ]:
# now we seek to append all the shopping cart items to our matrix dataframe

# create a counter and set the counter to 0 
n = 0

# iterate through array of shopping items
for i in items_clean_series:
    
    # iterate through each tuple element
    for tup in i:
        
        # locate the column name matching the shopping cart item and insert the quantity order
        items_matrix.loc[n,tup[0]] = tup[1]
    
    # increase counter
    n+=1

# replace all Na values with 0
items_matrix.fillna(0, inplace = True)

In [ ]:
# append the orde_price for all of these cobinations of items and quantities
items_matrix['price'] = correct_rows['order_price']

In [ ]:
# manually check our result
items_matrix.head()

In [ ]:
# now we need to create a set of  multilinear equations to calculate individual item prices
linear_equation_set = items_matrix.head(10)

# save the names of items (10 items) as a list
col_names_matrix_a = linear_equation_set.columns.to_list()

# remove the last column name, price
col_names_matrix_a.pop()

# save these items and their names in a dataframe as part_a
part_a = linear_equation_set[col_names_matrix_a]

# save the price of these items in a Series as part_b
part_b = linear_equation_set['price']

In [ ]:
# use the to_numpy function to convert a dataframe to an array format
a_input = part_a.to_numpy(dtype ='int')

# use the to_numpy function to convert a dataframe to an array format
b_input = part_b.to_numpy(dtype ='int')

In [ ]:
# using the np.linalg.solve() function from the numpy library we are able to calculat the
# price of each item using multilinear equations
price_of_item_array = np.linalg.solve(a_input,b_input)

In [ ]:
# combine the linear_equation_set.columns and the price_of_item_array into one dictionary
price_of_item = dict(zip(linear_equation_set.columns,price_of_item_array))

# iterate through keys and value of the dictionary price_of_item
for key,val in price_of_item.items():
    
    # save the rounded item value for as a key
    price_of_item[key] = int(round(val))


In [ ]:
# calculate the order price of each shopping cart row value
def calc_order_price(items):
    
    # save the important values which is product name and quantity
    items_order = clean_input(items)
    
    # create an int variable for price 
    price = 0
    
    # itterate through the ordered items
    for i in items_order:
        
        # calculate the combined price of all the items in the shopping cart 
        # considering both the item price and quantity
        price = int(price + price_of_item[i[0]] * i[1])
        
    # return the overall price
    return price

In [ ]:
# function that checks if calculated and given order price match
def check_order_price(given,cal):
    
    # cheking if calculated and given order price are identical
    if given - cal == 0:
        return True
    
    # in all other cases return no
    else:
        return False

In [ ]:
# function that fixes the given order price
def fix_order_price(given,cal):
    
    # in case of the given price not deviating from the calculated price
    if given - cal == 0:
        
        # return the original price
        return given
    
    # in all other cases
    else:
        
        # return the calculated price
        return cal

In [ ]:
# apply the calc_order_price to all rows and save the result in a new column named 'correct_order_price'
dirty_data_df['correct_order_price'] = dirty_data_df.apply(lambda row: calc_order_price(row['shopping_cart']) ,axis=1)

In [ ]:
# apply the check_order_price to all rows 
# in cae a price deviates from the given price save it
incorrect_price_series = dirty_data_df.apply(lambda row: check_order_price(row['order_price'],row['correct_order_price']) ,axis=1)

In [ ]:
# check for many incorrect order_price there are
incorrect_price_series.value_counts()


In [ ]:
# append the index value of all the rows with faulty season values given the date to
# the list fixed_columns
[fixed_columns.append(x) for x in list(dirty_data_df.index[incorrect_price_series == False])]

In [ ]:
# using the function fix_order_price we calculate the correct order price for all rows and save it in the
# orginal 'order_price' column 
dirty_data_df['order_price'] = dirty_data_df.apply(lambda row: fix_order_price(row['order_price'],row['correct_order_price']) ,axis=1)

In [ ]:
# do final check if all order prices are correct 
incorrect_price_series = dirty_data_df.apply(lambda row: check_order_price(row['order_price'],row['correct_order_price']) ,axis=1)

# display count
incorrect_price_series.value_counts()

### B.6 Check the order_total price 

In [ ]:
# function that calculates the correct order_total value 
def correct_order_total(order_price, discount, delivery):
    
    # calculate the discount for each order
    discount = (100 - discount) * 0.01
    
    # calucalte the overall correct order_total 
    correct_total = round((order_price * discount) + delivery,2)
    
    # return new order total
    return correct_total

In [ ]:
# function that checks if the order total is correct
def check_order_total(order_total,correct_order_total):
    
    # check if calculated and given order_total are identical
    if order_total - correct_order_total == 0:
        
        # if yes return True value
        return True
    
    # in all other cases return False value
    else:
        return False
    
    

In [ ]:
# using the correct_order_total function calculate the correct_order_total and save it
# in a new column named 'correct_order_total'
dirty_data_df['correct_order_total'] = dirty_data_df.apply(lambda row: correct_order_total(row['order_price'],row['coupon_discount'],row['delivery_charges']) ,axis=1)

In [ ]:
# conduct final check if all 'order_price' values are now correct
correct_order_total_series = dirty_data_df.apply(lambda row: check_order_total(row['order_price'],row['correct_order_price']) ,axis=1)
correct_order_total_series.value_counts()

In [ ]:
# append the index value of all rows wwith incorrect 'order_price's
[fixed_columns.append(x) for x in list(dirty_data_df.index[correct_order_total_series == False])]

## <span style="color:blue"> C. Drop unwanted columns and write fully clean dataframe to csv file 

In [ ]:
# check all the columns
# to see which ones were added for calculation purposes and thus need to be dropped
dirty_data_df.head()

In [ ]:
# we drop all the unwanted columns
dirty_data_df.drop(['sentiment_score','cor_distance_to_nearest_warehouse', 'correct_order_price', 'correct_order_total'], axis = 1, inplace = True)

In [ ]:
# final check to see in all unwanted columns were droped 
dirty_data_df.head(10)

In [ ]:
# finally we write the cleaned dataframe to a csv file 
dirty_data_df.to_csv('26255367_dirty_data_solution.csv', index = False)

# <span style="color:red"> 2. Removing outliers  

In this dataframe we will locate and remove all outliers in the column delivery_charges.

The term outlier referes to data points that vary greatly from the preceived norm. Some outliers might be 'justified' since they represnent certain contions, for example a very long delivery distance in the case of our delivery_charges. Thus, to access which outliers are justified and which aren't we created a Linear Regression Model for each season. The steps to identify ouliers and fix them are outlined below.

### 2.1 Read in data from csv file

In [ ]:
# read in csv file with outlier data
outlier_data_df = pd.read_csv('26255367_outlier_data.csv')

outlier_data_df.head()

### 2.2 Perpare dataframe for outlier analysis

In [ ]:
# function that converts a boolean scale from True/Flase to 0/1
def convert_boolean(series): 
    
    # if value = True we repalce it with 1
    if True: 
        
        # return value 1
        return 1 
    
    # if value = Flase we repalce it with 0
    else: 
        
        # return value 0
        return 0 

In [ ]:
# using the apply and above defined convert_boolean function we convert True or False 
# into 1 or to for quantitative calculation in linear regression 
outlier_data_df['is_happy_customer'] = outlier_data_df['is_happy_customer'].apply(lambda row: convert_boolean(row))
outlier_data_df['is_expedited_delivery'] = outlier_data_df['is_expedited_delivery'].apply(lambda row: convert_boolean(row))

### 2.3 Plot delivery charges in boxplot

In [ ]:
# we create a boxplot of distribution of delivery charges grouped by season 
# outliers are observed in each season 
delivery_charges_outlier_bp = outlier_data_df.boxplot(column = 'delivery_charges', by = 'season')

### 2.4 Create Linear Regression Model for delivery_charges

In [ ]:
## Create Linear Regression Model to predict delivery charges for each season

# different season name 
seasons = {'Spring', 'Summer', 'Autumn', 'Winter'}

# x variable for the multiple regression model 
# y variable is the target value we predict using the regression model 
X_var = ['is_happy_customer','is_expedited_delivery','distance_to_nearest_warehouse']
Y_var = ['delivery_charges']

# create list to save each season in
seperated_df_by_season = []

# loop through the season name 
for season in seasons: 

    # filter out the rows whose season name is the same as the loop 
    season = outlier_data_df[outlier_data_df.season == season]
    
    # instanciate linear regression function and fit the x and y variable specified above
    regression_season = LinearRegression().fit(season[X_var], season[Y_var])
    
    # calculate the predicted delivery_charges based on the model built 
    # append the values to 'normalised_delivery_charges' column in each season 
    season['normalised_delivery_charges']= regression_season.predict(season[X_var])
    
    # the difference between the true value and the predicted value from the model above 
    season['residuals'] = season['delivery_charges'] - season['normalised_delivery_charges']
    
    
    # after calculating the residuals and predicted values with the linear regression model, 
    # append the data frame of each season into a list for concatenation at the end
    seperated_df_by_season.append(season)
    

### 2.5 Plot the  delivery_charges to get overview of data spread

In [ ]:
## Plot the calculated delivery charges for each season using histograms 

# plot spring charges 
fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
n, bins, patches = ax1.hist(seperated_df_by_season[0]['delivery_charges'], color='r')
ax1.set_xlabel('Delivery Charges')
ax1.set_ylabel('Frequency')
ax1.set_title('Spring')

# plot summer charges 
fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)
n, bins, patches = ax2.hist(seperated_df_by_season[1]['delivery_charges'])
ax2.set_xlabel('Delivery Charges')
ax2.set_ylabel('Frequency')
ax2.set_title('Summer')

# plot autumn charges 
fig3 = plt.figure()
ax3 = fig3.add_subplot(1, 1, 1)
n, bins, patches = ax3.hist(seperated_df_by_season[2]['delivery_charges'], color='black')
ax3.set_xlabel('Delivery Charges')
ax3.set_ylabel('Frequency')
ax3.set_title('Autumn')

# plot winter charges 
fig4 = plt.figure()
ax4 = fig4.add_subplot(1, 1, 1)
n, bins, patches = ax4.hist(seperated_df_by_season[3]['delivery_charges'], color='green')
ax4.set_xlabel('Delivery Charges')
ax4.set_ylabel('Frequency')
ax4.set_title('Winter')

We can also plot the delivery charges of each season using bloxplots. This helps use to easily visualize outliers (above and blow the upper and lower whisker).

In [ ]:
# plot the boxplot for spring delivery charges
ax_spring = seperated_df_by_season[0].boxplot('residuals', figsize = (10,6))
ax_spring.set_title('Residual Plot: Spring')

In [ ]:
# plot the boxplot for summer delivery charges
ax_summer = seperated_df_by_season[1].boxplot('residuals', figsize = (10,6))
ax_summer.set_title('Residual Plot: Summer')

In [ ]:
# plot the boxplot for autumn delivery charges
ax_autumn = seperated_df_by_season[2].boxplot('residuals', figsize = (10,6))
ax_autumn.set_title('Residual Plot: Autumn')

In [ ]:
# plot the boxplot for winter delivery charges
ax_winter = seperated_df_by_season[3].boxplot('residuals', figsize = (10,6))
ax_winter.set_title('Residual Plot: Winter')

### 2.6 Compute upper and lower whiskers to determine outliers

After plotting the various delivery charges we can see that there outliers in each season.

In [ ]:
# function that calculates iqr and the whiskers given the q1 and q3 values as input
def compute_whiskers(Q1, Q3):
    
    # get iqr
    IQR = Q3 - Q1
    
    # get lower whisker 
    lower_w = Q1 - 1.5 * IQR
    
    # get upper whisker 
    upper_w = Q3 + 1.5 * IQR
    
    # return the iqr and both whiskers
    return IQR, lower_w, upper_w


### 2.7 Determine outliers and remove them

In [ ]:
## Calculate the outliers and remove them 

# looping through each data frame in seperated_df_by_season list
for season in seperated_df_by_season: 
    
    # the first quartile, 1/4th of the data points 
    Q1_season = season['residuals'].quantile(0.25)
    
    # the third quartile, 3/4 of the data points 
    Q3_season = season['residuals'].quantile(0.75)
    
    # IQR, lower whisker, and upper whisker returned by compute_whiskers function
    IQR_season, lower_season, upper_season = compute_whiskers(Q1_season, Q3_season)
    
    # if the value is less than the lower whisker or higher than the upper whisker, 
    # drop the rows
    season.drop(season[(season.residuals < lower_season) |(season.residuals > upper_season)].index,\
                inplace=True)
     

In [ ]:
# pd.concat() concatenate all the data frame in seperated_df_by_season list 
# creating boxplot of the delivery_charges grouped by each season 
# outliers are still detected after the first round of outlier removal
# repeating the same procedure to remove another round of outliers 
pd.concat(seperated_df_by_season).boxplot(column = 'delivery_charges', by = 'season')


In [ ]:
## Calculate the outliers again and remove the remaining ones 

seperated_df_by_season_follow_up =[]

# looping through each data frame in seperated_df_by_season list
for season in seperated_df_by_season:
    
    regression_season = LinearRegression().fit(season[X_var], season[Y_var])
    
    season['normalised_delivery_charges']= regression_season.predict(season[X_var])
    
    season['residuals'] = season['delivery_charges'] - season['normalised_delivery_charges']
    
    Q1_season = season['residuals'].quantile(0.25)
    
    Q3_season = season['residuals'].quantile(0.75)
    
    IQR_season, lower_season, upper_season = compute_whiskers(Q1_season, Q3_season)
    
    
    # if the value is less than the lower whisker or higher than the upper whisker, drop the rows
    season.drop(season[(season.residuals < lower_season) |(season.residuals > upper_season)].index, inplace=True)
    
    seperated_df_by_season_follow_up.append(season)

In [ ]:
# do final check to see if there are no more outliers in delivery_charges detected 
pd.concat(seperated_df_by_season_follow_up).boxplot(column = 'delivery_charges', by = 'season')

Edit was successfull and we could not detect more outliers in delivery_charges.

In [ ]:
# concatenate all the dataframes in list seperated_df_by_season_follow_up
outlier_removal_solution = pd.concat(seperated_df_by_season_follow_up)

# sorting based on original index
outlier_removal_solution.sort_index(inplace = True)

# display how many outliers were detected and removed alltogether 
# in the column delivery_charges
outlier_data_df.shape[0]-outlier_removal_solution.shape[0]

### 2.8 Drop unwanted columns and write clean dataframe to csv file 

In [ ]:
# check the columns in outlier df and identify unwanted ones
outlier_removal_solution.head()

In [ ]:
# notice that 'normalised_delivery_charges','residuals' are no longer needed and thus
# can be dropped
# now drop all the unwanted columns
outlier_removal_solution.drop(['normalised_delivery_charges','residuals'], axis = 1, inplace = True)

In [ ]:
outlier_removal_solution.to_csv('26255367_outlier_data_solution.csv',index = False)

# <span style="color:red"> 3. Imputing missing values 

In this dataframe we will locatye and fix all 'coverage' errors. To impute all the missing values we need to a) find the columns affected by missing values and b) find methods to impute such missing values.

### 3.1 Read in data from csv file

In [ ]:
# read in our missing dataf file in csv format
missing_data_df = pd.read_csv('26255367_missing_data.csv')

In [ ]:
# display first 10 rows
missing_data_df.head(10)

Currently missing values have NaN value inserted.

### 3.2 Find all columns with missing values

In [ ]:
# shows dimensions of the dataframe
missing_data_df.shape


In [ ]:
# shows information of all columns
missing_data_df.info()


After this check we know that there are missing values (NaN values) in 6 columns, which are:

- nearest_warehouse
- order_price
- delivery_charges
- order_total
- distance_to_nearest_warehouse
- is_happy_customer

For all of these columns we will now impute the missing values based on the surrounding information.

### 3.3 Impute missing is_happy_customer values

Values in the is_happy_customer column can only take a boolean value (True = 1, False = 0). We will determine the is_happy_customer based on the review the customer left. Thus, we can reuse the function created in the first part of the assignment where we edited the dirty_data.

In [ ]:
# apply the function get_sentiment to the column latest_customer_review to
# calculate a sentiment score
# that sentiment score is then saved in a new column called 
# missing_data_df['sentiment_score']
missing_data_df['sentiment_score'] = missing_data_df.apply(lambda row: get_sentiment(row['latest_customer_review'])["compound"],axis=1)

In [ ]:
missing_data_df.head()

In [ ]:
# function to fix the is_happy_customer boolean values 
def fix_happy_missing_val(score):
    
    # return happy True if sentiment_score >= 0.05
    if score  >= 0.05: 
        return 1.0
    
    # return happy False in all other case
    else:
        return 0.0

In [ ]:
# now we imput all the missing is_happy_customer values values
# we fill them by applying the fix_happy_missing_val methode to missing_data_df['is_happy_customer'] column
missing_data_df['is_happy_customer'] = missing_data_df.apply(lambda row: fix_happy_missing_val(row['sentiment_score']),axis=1)

In [ ]:
# conduct a final check if all the missing values were calculate
missing_data_df['is_happy_customer'].value_counts()

The final check showed that there are now 500 values in the missing_data_df['is_happy_customer'] column.

### 3.4 Impute missing nearest_warehouse values

To impute the missing values for nearest_warehouse we will use the read in coordinate information of of the warehouses and compare them with the customer's coordinates. The we will calculate the distance to each warehouse and choose the shortest distance.

In [ ]:
# read in the names of the three warehouse 
warehouse_df = pd.read_csv('warehouses.csv')

warehouse_df

In [ ]:
# calculate which warehouse is the nearest warehouse to the customer
# done using the compare_dist_warehouse() function that was defined in part 1 of the assignment
missing_data_df['nearest_warehouse'] = missing_data_df.apply(lambda row: compare_dist_warehouse(row['customer_lat'],row['customer_long']),axis=1)

In [ ]:
# conduct a final check if all the missing values were calculate
missing_data_df['nearest_warehouse'].value_counts()

The final check showed that there are now 500 values in the missing_data_df['nearest_warehouse'] column.

### 3.5 Impute missing distance_to_nearest_warehouse values

To impute the missing values for distance_to_nearest_warehouse we will use the read in coordinate information of of the warehouses and customer's coordinates. Then we will calculate the distance to the warehouse using the cal_warehouse_dist function defined in the first part of the assignmnet .

In [ ]:
# calculate the correct distance from a customer to the nearest warehouse and save that value
# column missing_data_df['distance_to_nearest_warehouse']
missing_data_df['distance_to_nearest_warehouse'] = missing_data_df.apply(lambda row: cal_warehouse_dist(row['nearest_warehouse'],row['customer_lat'],row['customer_long']),axis=1)

In [ ]:
# conduct a final check if all the missing values were calculate
missing_data_df['distance_to_nearest_warehouse'].isnull().value_counts()

In [ ]:
missing_data_df.head()

The final check showed that there are now 500 values in the missing_data_df['distance_to_nearest_warehouse'] column.

### 3.7 Impute missing order_price values

To impute the missing values for order_price we will use the calc_order_price() function defined in part one of 
the assignment. This function imputes missing values for order_price using the items and their quantity in the column 'shopping_cart.'

In [ ]:
# apply the calc_order_price to all rows in column 'shopping_cart' and save the result
# in missing_data_df['order_price']
missing_data_df['order_price'] = missing_data_df.apply(lambda row: calc_order_price(row['shopping_cart']) ,axis=1)

In [ ]:
# conduct a final check if all the missing values were calculate
missing_data_df['order_price'].isnull().value_counts()

The final check showed that there are now 500 values in the missing_data_df['order_price'] column.

### 3.8 Impute missing order_total values

To impute the missing values for order_total we can use the realtionship that exists between the columns:

- order_price
- delivery_charges
- order_total

The column order_price represents the combined price of all ordere items PRIOR to applying any discounts or adding
delivery fees. Since order_total values are derived by adding together the order_price with applied coupon_discount and the delivery_charges we can calculate the order_total values using the order_price, coupon_discount, and delivery_charges. This realtionship between the four variables can be expresed with the following formula:

order_price * ((100 - coupon_discount) / 100) + delivery_charges = order_total

In order for this formular to work we need to check that in the case of order_total values missing the 
other values are present. We know that the values for order_price and coupon_discount are complete. Thus, we only need to check whether in the rows where order_total values are missing the delivery_charges are present.

In [ ]:
# check that all required values for order_total value calculation are present
# checking overlap using isnull() isin() and .index[] functions
missing_data_df.index[missing_data_df.delivery_charges.isnull()].isin(set(missing_data_df.index[missing_data_df.order_total.isnull()]))

We confirmed that in the rows where order_total values are missing the delivery_charges are present.

In [ ]:
# function that calculates the total_oder price based on the relationship between
# order_price, delivery_charges, and order_total

def calc_order_total(order_price, discount, delivery_charges):
    
    # apply the above defined formula
    total_order_price = round(order_price * ((100-discount)/100) + delivery_charges,2)
    
    # return the calculated total_order value
    return total_order_price

In [ ]:
# select all the rows where the order_total value is missing
# and save it as a list 
total_order_miss_row = missing_data_df[missing_data_df.order_total.isnull()].index.to_list()

In [ ]:
# apply the calc_order_total to all rows in column 'order_total' and save the result
# in missing_data_df['order_price']
missing_data_df.iloc[total_order_miss_row,missing_data_df.columns.get_loc('order_total')] = missing_data_df.iloc[total_order_miss_row].apply(lambda row: calc_order_total(row['order_price'],row['coupon_discount'],row['delivery_charges']) ,axis=1)

In [ ]:
# check if any missing values for order_total remain
missing_data_df.info()

The final check showed that there are now 500 values in the missing_data_df['order_total'] column.

### 3.9 Impute missing delivery_charges values

To impute the missing values for delivery_charges we can use the realtionship that exists between the columns:

- order_price
- delivery_charges
- order_total

The column order_price represents the combined price of all ordere items PRIOR to applying any discounts or adding
delivery fees. Since order_total values are derived by adding together the order_price with applied coupon_discount and the delivery_charges we can calculate the order_total values using the order_price, coupon_discount, and delivery_charges. This realtionship between the four variables can be expresed with the following formula:

order_total - order_price * ((100 - coupon_discount) / 100) = delivery_charges

In order for this formular to work we need to check that in the case of order_total values missing the 
other values are present. We know that the values for order_price and coupon_discount are complete. Thus, we only need to check whether in the rows where order_total values are missing the delivery_charges are present.

In [ ]:
# check that all required values for  delivery_charges value calculation are present
# checking overlap using isnull() isin() and .index[] functions
missing_data_df.index[missing_data_df.delivery_charges.isnull()].isin(set(missing_data_df.index[missing_data_df.order_total.isnull()]))

We confirmed that in the rows where order_total values are missing the delivery_charges are present.

In [ ]:
# select all the rows where the delivery_charges value is missing
# and save it as a list 
delivery_charges_miss_row = missing_data_df[missing_data_df.delivery_charges.isnull()].index.to_list()

In [ ]:
delivery_charges_miss_row

In [ ]:
# function that calculates the delivery charges based on
# order_price, discount, and order_total
def calc_delivery_charges(total_order_price, order_price, discount):
    
    # apply the above defined formula
    delivery_charges = round(total_order_price - order_price * ((100-discount)/100), 2)

    # return the calculated delivery_charges 
    return delivery_charges

In [ ]:
# apply the calc_order_total to all rows in column 'order_total' and save the result
# in missing_data_df['order_price']
missing_data_df.iloc[delivery_charges_miss_row,missing_data_df.columns.get_loc('delivery_charges')] = missing_data_df.iloc[delivery_charges_miss_row].apply(lambda row: calc_delivery_charges(row['order_total'],row['order_price'],row['coupon_discount']) ,axis=1)

In [ ]:
# check if any missing values for 'order_total' remain
missing_data_df.info()

The final check showed that there are now 500 values in the missing_data_df['delivery_chargesl'] column.

### 3.10 Drop unwanted columns and write final dataframe to csv file

In [ ]:
# check for unwanted columns 
missing_data_df.head(10)

In [ ]:
# we drop all the unwanted columns
missing_data_df.drop('sentiment_score', axis = 1, inplace = True)

In [ ]:
# check if we successfully dropped all unwanted columns 
missing_data_df.head(10)

In [ ]:
# finally we write the edited dataframe to a csv file 
missing_data_df.to_csv('26255367_missing_data_solution.csv', index = False)

# 4. Summary

In summary, I have been able to complete all three tasks:

- 'clean' inaccurate data
- remove outliers
- impute missing values 

Through out the assignment I had a couple of major take-ways.
Firstly, the combination of apply(), and lambda functions in combination with custome built functions proved to be the most efficient way to edit pandas dataframes. Secondly, an overarching technique I found to be very usefull, and that I applied to all of the wrangling tasks , was to do a 'check-up' after each step, to see if the desired changes were succefully implemented and I can move to the next step. Thirdly, one of the main benefits of finishing the cleaning dirty data task first, was that we could re-use functions and techniques applied in the first part for the thrid part where we imputed missing values. 

Overall the assignment was a great and fun exercise to get familiar with data wrangling techniques and pandas dataframe manipulations. 

## 5. References

- Since the tutorial material was very comprehensive I didn't need a lot of out-of-class materials, outside of the resources given in the assignment brief, general pandas, and python documentation.

https://docs.python.org/3/

https://pandas.pydata.org/docs/

https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula

https://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg

https://riptutorial.com/numpy/example/12987/solve-linear-systems-with-np-solve

https://www.kdnuggets.com/2019/06/select-rows-columns-pandas.html